In [2]:
import os
import pickle
from enum import IntEnum
import re
import sys
import json
#import sklearn

sys.path.insert(0, 'C:/Users/akomarla/OneDrive - NANDPS/Desktop/Repos/lad')

#sys.path.insert(0, os.path.join(os.path.dirname(sys.executable), '..', '..', '..'))  # LAD

from LAD.model import generate_data, LADModel, LADConfig, DataReductionTechnique
from LAD.markup import LogMarkup
from LAD.base import Corpus
import LAD.model as model

from sklearn.neighbors import LocalOutlierFactor
import hashlib


In [4]:
sys.path.insert(0, 'C:/Users/akomarla/OneDrive - NANDPS/Desktop/Repos/lad')

!pip install tensorflow

In [458]:
pip install pytorch_lightning

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install rich

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install drain3

Note: you may need to restart the kernel to use updated packages.


In [459]:
pip install torchvision 

     ---------------------------------------- 1.2/1.2 MB 3.6 MB/s eta 0:00:00
     -------------------------------------- 172.3/172.3 MB 3.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
def preprocess(corpus):
    ret_corpus = []
    og_corpus = []
    for logfile in corpus:
        lines = []
        og_lines = []
        for line in logfile:
            og = line
            
            regex = r'^((?:\[[-A-Za-z]+\]\s+?)?(?:\[\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2}[-\s]+[-A-Za-z\s]+\])?)'
            t = re.sub(regex, ' ', line)
            # <01/12/23-20:33:25>
            t = re.sub(r'<\d{2}/\d{2}/\d{2}-\d{2}:\d{2}:\d{2}>', ' ', t)
            t = re.sub(r'\s+', ' ', t).lower().strip()
            # If the line was junk, skip it
            if not t:
                continue
            lines.append(t)
            og_lines.append(og)
        ret_corpus.append(lines)
        og_corpus.append(og_lines)
    return ret_corpus, og_corpus

In [4]:
# Uart logs (sanity config)
# train_file_path = '/Users/akomarla/OneDrive - NANDPS/Desktop/Repos/lad/examples/data/sanity_config_pos.txt'
# val_file_path = '/Users/akomarla/OneDrive - NANDPS/Desktop/Repos/lad/examples/data/sanity_config_val.pkl'
# failed_path = '/Users/akomarla/OneDrive - NANDPS/Desktop/Repos/lad/examples/data/sanity_config_fail.txt'

# Console text (sanity config)
train_file_path = '/Users/akomarla/OneDrive - NANDPS/Desktop/Repos/lad/examples/data/sanity_ct_train.txt'
val_file_path = '/Users/akomarla/OneDrive - NANDPS/Desktop/Repos/lad/examples/data/sanity_ct_val.txt'
failed_path = '/Users/akomarla/OneDrive - NANDPS/Desktop/Repos/lad/examples/data/sanity_ct_fail.txt'

train = True
val = False

gen_val_path = './val.txt'
gen_train_path = './train.txt'

# Load a corpus of data for training
if train:
    with open(train_file_path, 'r', encoding="utf8") as f:
        corpus = json.load(f)
        corpus, _  = preprocess(corpus)

# Load a corpus of data for validation
if val:
    with open(val_file_path, 'r', encoding="utf8") as f:
        corpus = json.load(f)
        val_corpus, _ = preprocess(corpus)

# Generate data
config = LADConfig(False)
config.device = 'cuda'  # You can manually set config attributes
config.hidden_size = 128
config.window_size = 10
vocab, _ = generate_data(gen_train_path, corpus, config, reduction=DataReductionTechnique.LOG, remove_single_tokens=True, reduction_cap=10)


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 23.15it/s]
40it [00:01, 26.90it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 86.62it/s]


In [5]:
vocab.__dict__['_freq']

defaultdict(int,
            {'started by upstream project <*> build number <*>': 80,
             'originally caused by:': 80,
             '<a href="https://npsg-bit.elements.local/projects/fsedev/repos/trenton/pull-requests/10009" >pr #10009 feature/nsgse-226085-io-manager-to-drive-state </a>': 2,
             'running as system': 40,
             '- loading node environment variables.': 40,
             'building remotely on <*> <*> <*> nand_b3mb <*> <*> <*> <*> in workspace /home/jenkins/jenkins_jobs/workspace/prod_bhb_ci_b3mb_hw_spdk_branch/cfg_file/sanity.cfg/iteration/1/nand/b3mb/label/nand_b3mb': 40,
             'deleting project workspace...': 80,
             'deferred wipeout is disabled by the job configuration...': 80,
             'done': 56,
             'run condition <*> <*> <*> enabling <*> for step [execute shell]': 120,
             'new run name is <*>': 80,
             '[current build status] check if current [success] is worse or equals then [success] and bett

In [536]:
def gen_vocab_key_map(vocab, trunc):
    """
    takes in log vocab dict (log vocab str: count) and returns a new vocab dict (log vocab str: hash)
    """
    vocab_key_map = {}
    for val in vocab:
        result = hashlib.md5(val.encode())
        vocab_key_map[val] = result.hexdigest()[:trunc]
    return vocab_key_map

def gen_log_key_map(log_text, vocab_key_map):
    """
    takes in list of raw log strs (no template extraction) and maps them to the vocab followed by hash key
    returns a dict {log str: hash(log str_n)}
    """
    log_dict = {}
    count = len(log_text)
    for log_str in log_text:
        for val in vocab_key_map.keys():
            if all(word in log_str for word in val.replace('<*>', '').split()):  
                log_dict[log_str] = vocab_key_map[val]
                count = count - 1
                break
    return log_dict, count

def gen_corpus_key(log_text, vocab_key_map):
    """
    takes in list of raw log strs (no template extraction) and maps them to the vocab followed by hash key
    returns a list of hashes in the order corresponding to the log strs [hash(log str_n),...]
    """
    log_key_list = []
    count = len(log_text)
    for log_str in log_text:
        for val in vocab_key_map.keys():
            if all(word in log_str for word in val.replace('<*>', '').split()):  
                log_key_list.append(vocab_key_map[val])
                count = count - 1
                break
    return log_key_list, count

def gen_seq(log_text, time_window, vocab_key_map, how, join):
    """
    takes in list of log strs and pools them into sequences based on time window or length
    returns dict {seq n : log str,...} or {seq n : hash(log str),...}
    """
    seq_dict = {}
    
    log_dict, _ = gen_log_key_map(log_text = log_text, 
                                  vocab_key_map = vocab_key_map)
    if how == 'str':
        for i in range(len(log_text)-time_window):
            if join:
                seq_dict['seq_'+str(i)] = " ".join(log_text[i:i+time_window])
            else: 
                seq_dict['seq_'+str(i)] = log_text[i:i+time_window]
    elif how == 'key':
        for i in range(len(log_text)-time_window):
            if join:
                seq_dict['seq_'+str(i)] = " ".join([log_dict.get(t) for t in log_text[i:i+time_window]])
            else:
                seq_dict['seq_'+str(i)] = [log_dict.get(t) for t in log_text[i:i+time_window]]
    return seq_dict


In [83]:
vocab_hash = gen_vocab_key_map(vocab = vocab.__dict__['_freq'], trunc = None)
vocab_hash

{'started by upstream project <*> build number <*>': 'ec4d74cfeab266bbef483f195cc6ccb2',
 'originally caused by:': 'e00d12dc9209b2120eb86e4d34784989',
 '<a href="https://npsg-bit.elements.local/projects/fsedev/repos/trenton/pull-requests/10009" >pr #10009 feature/nsgse-226085-io-manager-to-drive-state </a>': 'fabfcd33b09de18170854944000c006e',
 'running as system': '855d319d7511483476811044aa8b18d6',
 '- loading node environment variables.': 'c41e6c4bbfb734d1843f046aac42a5f3',
 'building remotely on <*> <*> <*> nand_b3mb <*> <*> <*> <*> in workspace /home/jenkins/jenkins_jobs/workspace/prod_bhb_ci_b3mb_hw_spdk_branch/cfg_file/sanity.cfg/iteration/1/nand/b3mb/label/nand_b3mb': 'e104aa278623a0979a578f81afc1f309',
 'deleting project workspace...': '870453d9c0cc9ef5e822970a36e324f8',
 'deferred wipeout is disabled by the job configuration...': '5677d0e12fa5ad9d3f67ba2d6230be04',
 'done': '6b2ded51d81a4403d8a4bd25fa1e57ee',
 'run condition <*> <*> <*> enabling <*> for step [execute shell]':

In [84]:
log_dict, no_hash_count = gen_log_key_map(log_text = corpus[0][0:100], vocab_key_map = vocab_hash)
log_dict

{'started by upstream project "prod_bhb_ci_b3mb_hw_spdk_branch" build number 12376': 'ec4d74cfeab266bbef483f195cc6ccb2',
 'originally caused by:': 'e00d12dc9209b2120eb86e4d34784989',
 'started by upstream project "bhb_ci_main_branch" build number 18647': 'ec4d74cfeab266bbef483f195cc6ccb2',
 '<a href="https://npsg-bit.elements.local/projects/fsedev/repos/trenton/pull-requests/10009" >pr #10009 feature/nsgse-226085-io-manager-to-drive-state </a>': 'fabfcd33b09de18170854944000c006e',
 'running as system': '855d319d7511483476811044aa8b18d6',
 '- loading node environment variables.': 'c41e6c4bbfb734d1843f046aac42a5f3',
 'building remotely on f223-508-11-sm (ssdp42kx040t1es build_a3 nand_b3mb gen4_b3mb conval_rack_f223_508 bhb_4tb asic_c0) in workspace /home/jenkins/jenkins_jobs/workspace/prod_bhb_ci_b3mb_hw_spdk_branch/cfg_file/sanity.cfg/iteration/1/nand/b3mb/label/nand_b3mb': 'e104aa278623a0979a578f81afc1f309',
 'deleting project workspace...': '870453d9c0cc9ef5e822970a36e324f8',
 'deferr

In [537]:
seq_hash = gen_seq(log_text = corpus[0], time_window = 9, vocab_key_map = vocab_hash, how = 'key', join = True)
seq_hash

{'seq_0': 'ec4d74cfeab266bbef483f195cc6ccb2 e00d12dc9209b2120eb86e4d34784989 ec4d74cfeab266bbef483f195cc6ccb2 e00d12dc9209b2120eb86e4d34784989 fabfcd33b09de18170854944000c006e 855d319d7511483476811044aa8b18d6 c41e6c4bbfb734d1843f046aac42a5f3 e104aa278623a0979a578f81afc1f309 870453d9c0cc9ef5e822970a36e324f8',
 'seq_1': 'e00d12dc9209b2120eb86e4d34784989 ec4d74cfeab266bbef483f195cc6ccb2 e00d12dc9209b2120eb86e4d34784989 fabfcd33b09de18170854944000c006e 855d319d7511483476811044aa8b18d6 c41e6c4bbfb734d1843f046aac42a5f3 e104aa278623a0979a578f81afc1f309 870453d9c0cc9ef5e822970a36e324f8 5677d0e12fa5ad9d3f67ba2d6230be04',
 'seq_2': 'ec4d74cfeab266bbef483f195cc6ccb2 e00d12dc9209b2120eb86e4d34784989 fabfcd33b09de18170854944000c006e 855d319d7511483476811044aa8b18d6 c41e6c4bbfb734d1843f046aac42a5f3 e104aa278623a0979a578f81afc1f309 870453d9c0cc9ef5e822970a36e324f8 5677d0e12fa5ad9d3f67ba2d6230be04 6b2ded51d81a4403d8a4bd25fa1e57ee',
 'seq_3': 'e00d12dc9209b2120eb86e4d34784989 fabfcd33b09de1817085494400

In [90]:
corpus_hashed, _ = gen_corpus_key(log_text = corpus[0], vocab_key_map = vocab_hash)
corpus_hashed

['ec4d74cfeab266bbef483f195cc6ccb2',
 'e00d12dc9209b2120eb86e4d34784989',
 'ec4d74cfeab266bbef483f195cc6ccb2',
 'e00d12dc9209b2120eb86e4d34784989',
 'fabfcd33b09de18170854944000c006e',
 '855d319d7511483476811044aa8b18d6',
 'c41e6c4bbfb734d1843f046aac42a5f3',
 'e104aa278623a0979a578f81afc1f309',
 '870453d9c0cc9ef5e822970a36e324f8',
 '5677d0e12fa5ad9d3f67ba2d6230be04',
 '6b2ded51d81a4403d8a4bd25fa1e57ee',
 '150ffa0a18605ed01e34bff276f8901f',
 'a5b55a51018fa23ccec31e0309192dc9',
 '41c6f0c33fafcf3d9688d9186c5174e7',
 '150ffa0a18605ed01e34bff276f8901f',
 'f2301b91333f5fcea36348856c3ee1a3',
 'dd1e1bcc3a7469de81e846d112a20b66',
 '3d7dfd40b6d361e403cfce8735724f65',
 'e393483a5ae8c2f5e9524e4e3b7c1d23',
 '443e56d2646a8b34cc4dd84fdf165619',
 '76c238c0473afcd0eca71a29271e5459',
 '4ebfc3693af02e814d81204ab93d1322',
 '4ebfc3693af02e814d81204ab93d1322',
 '4ebfc3693af02e814d81204ab93d1322',
 '13aa5a3a678011ebe591d856fa995fd0',
 '13aa5a3a678011ebe591d856fa995fd0',
 '6b5806e659fdbf28047231a5df8c09d7',
 

In [408]:
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, decoders, trainers, processors
from tqdm.auto import tqdm

#MIN_FREQ = 0 # words appearing fewer than 3 times are treated as 'unknown'
unk_token = '[UNK]'
pad_token = '[PAD]'
sep_token = '[SEP]'
cls_token = '[CLS]'

bert_tokenizer = Tokenizer(models.WordLevel(unk_token=unk_token))
bert_tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
bert_tokenizer.normalizer = normalizers.Lowercase()

trainer = trainers.WordLevelTrainer(special_tokens=[pad_token, unk_token, sep_token, cls_token])
bert_tokenizer.train_from_iterator(tqdm(corpus_hashed), trainer=trainer)

if not os.path.exists('./bert_tokenizer'):
    os.mkdir('./bert_tokenizer')
bert_tokenizer.model.save('bert_tokenizer')

  0%|          | 0/1114 [00:00<?, ?it/s]

['bert_tokenizer\\vocab.json']

In [332]:
#tokenizer config
tokenizer_config = {"max_len": 512, "name_or_path": "bert_tokenizer"}

with open("bert_tokenizer/config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)

In [402]:
from transformers import BertTokenizer
bert_tokenizer = BertTokenizer.from_pretrained('./bert_tokenizer/vocab.json')
#bert_tokenizer.post_processor = processors.BertProcessing(
    #('[SEP]', bert_tokenizer.encode('SEP').ids[0]), ('[CLS]', bert_tokenizer.encode('CLS').ids[0]))

AttributeError: type object 'tokenizers.models.WordLevel' has no attribute 'from_pretrained'

In [548]:
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
from transformers import AutoTokenizer

tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)

tokenizer.train_from_iterator( 
    tqdm(corpus_hashed),
    vocab_size=30_000, 
    min_frequency=0,
    limit_alphabet=1000, 
    wordpieces_prefix='##',
    special_tokens=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']
    )

if not os.path.exists('./logbert'):
    os.mkdir('./logbert')
tokenizer.save_model('./logbert/')
tokenizer = BertTokenizer.from_pretrained('./logbert/vocab.txt', local_files_only=True)

  0%|          | 0/1114 [00:00<?, ?it/s]

loading file ./logbert/vocab.txt from cache at ./logbert/vocab.txt


In [549]:
test_string = ['ec4d74cfeab266bbef483f195cc6ccb2', 'c51df5ba4c18a', 'c51df5ba4c18a8dff10c6e315490da39', 'ec4d74cfeab266bbef483f195cc6ccb2']
input_ids = tokenizer.encode(test_string)
output = tokenizer.decode(input_ids)

In [558]:
train_text = [seq for seq in seq_hash.values()]

In [560]:
tokens_train = tokenizer.batch_encode_plus(train_text)

In [581]:
tokens_train

{'input_ids': [[1, 2357, 2275, 2357, 2275, 4299, 4255, 4283, 4295, 2344, 2], [1, 2275, 2357, 2275, 4299, 4255, 4283, 4295, 2344, 2339, 2], [1, 2357, 2275, 4299, 4255, 4283, 4295, 2344, 2339, 1666, 2], [1, 2275, 4299, 4255, 4283, 4295, 2344, 2339, 1666, 1668, 2], [1, 4299, 4255, 4283, 4295, 2344, 2339, 1666, 1668, 2350, 2], [1, 4255, 4283, 4295, 2344, 2339, 1666, 1668, 2350, 4238, 2], [1, 4283, 4295, 2344, 2339, 1666, 1668, 2350, 4238, 1668, 2], [1, 4295, 2344, 2339, 1666, 1668, 2350, 4238, 1668, 2359, 2], [1, 2344, 2339, 1666, 1668, 2350, 4238, 1668, 2359, 4154, 2], [1, 2339, 1666, 1668, 2350, 4238, 1668, 2359, 4154, 4200, 2], [1, 1666, 1668, 2350, 4238, 1668, 2359, 4154, 4200, 4292, 2], [1, 1668, 2350, 4238, 1668, 2359, 4154, 4200, 4292, 1671, 2], [1, 2350, 4238, 1668, 2359, 4154, 4200, 4292, 1671, 4249, 2], [1, 4238, 1668, 2359, 4154, 4200, 4292, 1671, 4249, 1655, 2], [1, 1668, 2359, 4154, 4200, 4292, 1671, 4249, 1655, 1655, 2], [1, 2359, 4154, 4200, 4292, 1671, 4249, 1655, 1655, 165

In [562]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])

In [563]:
train_mask

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])

In [564]:
train_seq

tensor([[   1, 2357, 2275,  ..., 4295, 2344,    2],
        [   1, 2275, 2357,  ..., 2344, 2339,    2],
        [   1, 2357, 2275,  ..., 2339, 1666,    2],
        ...,
        [   1, 2359,  318,  ..., 4085, 2344,    2],
        [   1,  318, 4051,  ..., 2344, 2339,    2],
        [   1, 4051,  318,  ..., 2339, 1666,    2]])

In [569]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_data = TensorDataset(train_seq, train_mask)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

In [584]:
train_data.__dict__

{'tensors': (tensor([[   1, 2357, 2275,  ..., 4295, 2344,    2],
          [   1, 2275, 2357,  ..., 2344, 2339,    2],
          [   1, 2357, 2275,  ..., 2339, 1666,    2],
          ...,
          [   1, 2359,  318,  ..., 4085, 2344,    2],
          [   1,  318, 4051,  ..., 2344, 2339,    2],
          [   1, 4051,  318,  ..., 2339, 1666,    2]]),
  tensor([[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          ...,
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]]))}

In [570]:
from transformers import *
model_config = BertConfig(vocab_size=len(vocab_hash), max_position_embeddings=20)
model = BertForMaskedLM(config=model_config)

In [571]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.2)

In [572]:
data_collator

DataCollatorForLanguageModeling(tokenizer=PreTrainedTokenizer(name_or_path='./logbert/vocab.txt', vocab_size=4304, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), mlm=True, mlm_probability=0.2, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [573]:
training_args = TrainingArguments(
    output_dir='bert_model.json',          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=10,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=10, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,  # evaluation batch size
    logging_steps=1000,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=1000,
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

using `logging_steps` to initialize `eval_steps` to 1000
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [579]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=corpus[1],
)

In [580]:
trainer.train()

***** Running training *****
  Num examples = 35
  Num Epochs = 10
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 80
  Gradient Accumulation steps = 8
  Total optimization steps = 10
  Number of trainable parameters = 86047986


TypeError: 'DataLoader' object is not subscriptable

In [80]:
import torch

labels = torch.tensor([x.ids for x in batch])
mask = torch.tensor([x.attention_mask for x in batch])
# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)
# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)
# loop through each row in input_ids tensor (cannot do in parallel)
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = 3  # our custom [MASK] token == 3
#We have 10000 tokenized sequences, each containing 512 tokens.

input_ids.shape

AttributeError: 'str' object has no attribute 'ids'

In [ ]:
bag = [item for sentence in val_corpus[0] for item in sentence.split('.') if item != '']
bag_size = len(bag)
print(bag_size)

In [ ]:
import random

sentence_a = []
sentence_b = []
label = []

for paragraph in val_corpus[0]:
    sentences = [
        sentence for sentence in paragraph.split('.') if sentence != ''
    ]
    num_sentences = len(sentences)
    if num_sentences > 1:
        start = random.randint(0, num_sentences-2)
        # 50/50 whether is IsNextSentence or NotNextSentence
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(sentences[start+1])
            label.append(0)
        else:
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(bag[index])
            label.append(1)

In [ ]:
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=512, truncation=True, padding='max_length')


In [ ]:
inputs['labels'] = torch.LongTensor([label]).T